In [26]:
# Loading Dependencies
from path import Path
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#load VADER
analyzer = SentimentIntensityAnalyzer()

In [2]:
# Loading Data
data = Path('Resources/reddit.csv')
reddit_df = pd.read_csv(data)
reddit_df.head()

,title,score,subreddit,url,num_comments,body,date
0,UPVOTE so everyone sees we got SUPPORT,265029,wallstreetbets,https://i.redd.it/sgoqy8nyt2e61.png,11825,NaN,2021-01-29 00:40:34
1,GME YOLO update — Jan 28 2021,230844,wallstreetbets,https://i.redd.it/opzucppb15e61.png,23532,NaN,2021-01-29 08:06:23
2,CLASS ACTION AGAINST ROBINHOOD. Allowing peopl...,204920,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,18318,LEAVE ROBINHOOD. They dont deserve to make mon...,2021-01-29 00:49:11
3,GME YOLO update — Jan 27 2021 ----------------...,185949,wallstreetbets,https://i.redd.it/a309gkm5yxd61.png,15495,NaN,2021-01-28 08:15:35
4,Can we all take a moment and appreciate the Mo...,184517,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,7105,NaN,2021-01-28 11:57:32


In [3]:
# Checking DTypes
reddit_df.dtypes

title           object
score            int64
subreddit       object
url             object
num_comments     int64
body            object
date            object
dtype: object

In [4]:
# Look at subreddit value counts
subreddit = reddit_df.subreddit.value_counts()
subreddit

investing         987
stocks            985
wallstreetbets    953
Name: subreddit, dtype: int64

In [5]:
# Look at body value counts
body = reddit_df.body.value_counts()
body

Please use this thread to discuss your portfolio, learn of other stock tickers, and help out users by giving constructive criticism.\r\n\r\nWhy quarterly?  Public companies report earnings quarterly; many investors take this as an opportunity to rebalance their portfolios.  We highly recommend you do some reading:  A list of [relevant posts & book recommendations.](https://www.reddit.com/r/stocks/wiki/index#wiki_relevant_posts_.26amp.3B_book_recommendations)\r\n\r\nYou can find stocks on your own by using a scanner like your broker's or [Finviz.](https://finviz.com/screener.ashx)  To help further, here's a list of [relevant websites.](https://www.reddit.com/r/stocks/wiki/index#wiki_relevant_websites.2Fapps)\r\n\r\nIf you don't have a broker yet, see our [list of brokers](https://www.reddit.com/r/stocks/wiki/index#wiki_brokers_for_investing) or search old posts.  If you haven't started investing or trading yet, then setup your [paper trading.](https://www.reddit.com/r/stocks/wiki/index#

In [6]:
# Remove the 'body' column.
reddit_df.drop(['body'], axis=1, inplace=True)
reddit_df.head()

,title,score,subreddit,url,num_comments,date
0,UPVOTE so everyone sees we got SUPPORT,265029,wallstreetbets,https://i.redd.it/sgoqy8nyt2e61.png,11825,2021-01-29 00:40:34
1,GME YOLO update — Jan 28 2021,230844,wallstreetbets,https://i.redd.it/opzucppb15e61.png,23532,2021-01-29 08:06:23
2,CLASS ACTION AGAINST ROBINHOOD. Allowing peopl...,204920,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,18318,2021-01-29 00:49:11
3,GME YOLO update — Jan 27 2021 ----------------...,185949,wallstreetbets,https://i.redd.it/a309gkm5yxd61.png,15495,2021-01-28 08:15:35
4,Can we all take a moment and appreciate the Mo...,184517,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,7105,2021-01-28 11:57:32


In [7]:
# Remove rows that have at least 1 null value.
reddit_df.dropna()

,title,score,subreddit,url,num_comments,date
0,UPVOTE so everyone sees we got SUPPORT,265029,wallstreetbets,https://i.redd.it/sgoqy8nyt2e61.png,11825,2021-01-29 00:40:34
1,GME YOLO update — Jan 28 2021,230844,wallstreetbets,https://i.redd.it/opzucppb15e61.png,23532,2021-01-29 08:06:23
2,CLASS ACTION AGAINST ROBINHOOD. Allowing peopl...,204920,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,18318,2021-01-29 00:49:11
3,GME YOLO update — Jan 27 2021 ----------------...,185949,wallstreetbets,https://i.redd.it/a309gkm5yxd61.png,15495,2021-01-28 08:15:35
4,Can we all take a moment and appreciate the Mo...,184517,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,7105,2021-01-28 11:57:32
...,...,...,...,...,...,...
2920,DID WE MISS THE BOTTOM?! How are people this i...,348,stocks,https://www.reddit.com/r/stocks/comments/g1m6u...,283,2020-04-15 17:03:23
2921,Favorite Solar Stock(s)?,348,stocks,https://www.reddit.com/r/stocks/comments/jmct3...,176,2020-11-02 10:46:49
2922,"""NIO forms battery asset company""",352,stocks,https://www.reddit.com/r/stocks/comments/id8z3...,102,2020-08-20 23:02:48
2923,Amazon is building a $1.5 billion hub for its ...,349,stocks,https://www.reddit.com/r/stocks/comments/5rghg...,145,2017-02-02 03:35:04


In [8]:
# Checking Sentiment Scores
def sentiment_analyzer_scores(sentence):
    score = analyzer.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
    
sentiment_analyzer_scores('UPVOTE so everyone sees we got SUPPORT')

UPVOTE so everyone sees we got SUPPORT-- {'neg': 0.0, 'neu': 0.636, 'pos': 0.364, 'compound': 0.5319}


In [9]:
#Add VADER metrics to dataframe
reddit_df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in reddit_df['title']]

reddit_df['neg'] = [analyzer.polarity_scores(v)['neg'] for v in reddit_df['title']]

reddit_df['neu'] = [analyzer.polarity_scores(v)['neu'] for v in reddit_df['title']]

reddit_df['pos'] = [analyzer.polarity_scores(v)['pos'] for v in reddit_df['title']]

reddit_df.head()

,title,score,subreddit,url,num_comments,date,compound,neg,neu,pos
0,UPVOTE so everyone sees we got SUPPORT,265029,wallstreetbets,https://i.redd.it/sgoqy8nyt2e61.png,11825,2021-01-29 00:40:34,0.5319,0.000,0.636,0.364
1,GME YOLO update — Jan 28 2021,230844,wallstreetbets,https://i.redd.it/opzucppb15e61.png,23532,2021-01-29 08:06:23,0.4278,0.000,0.679,0.321
2,CLASS ACTION AGAINST ROBINHOOD. Allowing peopl...,204920,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,18318,2021-01-29 00:49:11,-0.5994,0.107,0.893,0.000
3,GME YOLO update — Jan 27 2021 ----------------...,185949,wallstreetbets,https://i.redd.it/a309gkm5yxd61.png,15495,2021-01-28 08:15:35,0.4278,0.000,0.841,0.159
4,Can we all take a moment and appreciate the Mo...,184517,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,7105,2021-01-28 11:57:32,0.6369,0.000,0.794,0.206


In [10]:
reddit_groups = reddit_df.groupby("subreddit")

In [11]:
# Grouping Vader Scores for each Subreddit
reddit_groups.mean()

,score,num_comments,compound,neg,neu,pos
subreddit,,,,,,
investing,1779.078014,424.946302,-0.009373,0.080139,0.842621,0.077234
stocks,1405.035533,311.643655,0.038948,0.060024,0.852797,0.087179
wallstreetbets,29348.439664,2753.270724,0.046143,0.066779,0.834423,0.098805


In [12]:
reddit_df = reddit_df[reddit_df["subreddit"]!="stocks"]
reddit_df.tail()

,title,score,subreddit,url,num_comments,date,compound,neg,neu,pos
1935,does anyone here have friends and family still...,898,investing,https://www.reddit.com/r/investing/comments/ef...,609,2019-12-26 05:47:26,-0.0258,0.143,0.717,0.139
1936,Jobs growth soars in November as payrolls surg...,905,investing,https://www.reddit.com/r/investing/comments/e6...,496,2019-12-07 00:41:05,0.3818,0.000,0.776,0.224
1937,"Yale economists argue that ""the most financial...",894,investing,https://www.reddit.com/r/investing/comments/en...,400,2020-01-13 05:35:24,0.0460,0.100,0.792,0.108
1938,"Amazon earnings beat: $6.04 per share, vs. $5....",897,investing,https://www.reddit.com/r/investing/comments/al...,152,2019-02-01 08:04:51,0.4404,0.000,0.642,0.358
1939,U.S. stocks plunge after report that former na...,890,investing,https://www.reddit.com/r/investing/comments/7g...,377,2017-12-02 03:34:52,0.3400,0.000,0.893,0.107


In [13]:
# Changing Subreddit values to be 1 if from WSB and 0 if not.
reddit_df['subreddit'] = reddit_df['subreddit'].apply(lambda x:1 if x == "wallstreetbets" else 0)

C:\Users\aboon\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
reddit_df.head()

,title,score,subreddit,url,num_comments,date,compound,neg,neu,pos
0,UPVOTE so everyone sees we got SUPPORT,265029,1,https://i.redd.it/sgoqy8nyt2e61.png,11825,2021-01-29 00:40:34,0.5319,0.000,0.636,0.364
1,GME YOLO update — Jan 28 2021,230844,1,https://i.redd.it/opzucppb15e61.png,23532,2021-01-29 08:06:23,0.4278,0.000,0.679,0.321
2,CLASS ACTION AGAINST ROBINHOOD. Allowing peopl...,204920,1,https://www.reddit.com/r/wallstreetbets/commen...,18318,2021-01-29 00:49:11,-0.5994,0.107,0.893,0.000
3,GME YOLO update — Jan 27 2021 ----------------...,185949,1,https://i.redd.it/a309gkm5yxd61.png,15495,2021-01-28 08:15:35,0.4278,0.000,0.841,0.159
4,Can we all take a moment and appreciate the Mo...,184517,1,https://www.reddit.com/r/wallstreetbets/commen...,7105,2021-01-28 11:57:32,0.6369,0.000,0.794,0.206


In [15]:
reddit_df.tail()

,title,score,subreddit,url,num_comments,date,compound,neg,neu,pos
1935,does anyone here have friends and family still...,898,0,https://www.reddit.com/r/investing/comments/ef...,609,2019-12-26 05:47:26,-0.0258,0.143,0.717,0.139
1936,Jobs growth soars in November as payrolls surg...,905,0,https://www.reddit.com/r/investing/comments/e6...,496,2019-12-07 00:41:05,0.3818,0.000,0.776,0.224
1937,"Yale economists argue that ""the most financial...",894,0,https://www.reddit.com/r/investing/comments/en...,400,2020-01-13 05:35:24,0.0460,0.100,0.792,0.108
1938,"Amazon earnings beat: $6.04 per share, vs. $5....",897,0,https://www.reddit.com/r/investing/comments/al...,152,2019-02-01 08:04:51,0.4404,0.000,0.642,0.358
1939,U.S. stocks plunge after report that former na...,890,0,https://www.reddit.com/r/investing/comments/7g...,377,2017-12-02 03:34:52,0.3400,0.000,0.893,0.107


In [ ]:
reddit_df

In [16]:
# Creating X and Y sets
y = reddit_df["subreddit"]
X = reddit_df.drop(columns=["subreddit","title","url","date","num_comments","score"])

In [17]:
# Breaking sets into train and test.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(1455, 4)

In [18]:
y.value_counts()

0    987
1    953
Name: subreddit, dtype: int64

In [19]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

In [20]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [24]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
print(results)

     Prediction  Actual
0             0       0
1             0       0
2             0       1
3             1       1
4             0       0
..          ...     ...
480           0       0
481           0       1
482           0       0
483           0       0
484           1       1

[485 rows x 2 columns]


In [25]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5092783505154639


# Optimization Attempt #1 - Add Additional Hidden Layers

In [33]:
X_train.shape[1]

4

In [34]:
# define model
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

#third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# check structure of model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                400       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 3,471
Trainable params: 3,471
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
46/46 [==============================] - 0s 842us/step - loss: 0.6982 - accuracy: 0.4393
Epoch 2/100
46/46 [==============================] - 0s 887us/step - loss: 0.6868 - accuracy: 0.5515
Epoch 3/100
46/46 [==============================] - 0s 841us/step - loss: 0.6802 - accuracy: 0.5749
Epoch 4/100
46/46 [==============================] - 0s 878us/step - loss: 0.6711 - accuracy: 0.6399
Epoch 5/100
46/46 [==============================] - 0s 843us/step - loss: 0.6623 - accuracy: 0.6285
Epoch 6/100
46/46 [==============================] - 0s 843us/step - loss: 0.6450 - accuracy: 0.6576
Epoch 7/100
46/46 [==============================] - 0s 863us/step - loss: 0.6594 - accuracy: 0.6334
Epoch 8/100
46/46 [==============================] - 0s 820us/step - loss: 0.6528 - accuracy: 0.6236
Epoch 9/100
46/46 [==============================] - 0s 887us/step - loss: 0.6475 - accuracy: 0.6329
Epoch 10/100
46/46 [==============================] - 0s 864us/step - loss: 0.6621 - accura

46/46 [==============================] - 0s 975us/step - loss: 0.6165 - accuracy: 0.6654
Epoch 82/100
46/46 [==============================] - 0s 1ms/step - loss: 0.6058 - accuracy: 0.6703
Epoch 83/100
46/46 [==============================] - 0s 864us/step - loss: 0.6289 - accuracy: 0.6448
Epoch 84/100
46/46 [==============================] - 0s 878us/step - loss: 0.6272 - accuracy: 0.6553
Epoch 85/100
46/46 [==============================] - 0s 931us/step - loss: 0.6385 - accuracy: 0.6456
Epoch 86/100
46/46 [==============================] - 0s 932us/step - loss: 0.6124 - accuracy: 0.6687
Epoch 87/100
46/46 [==============================] - 0s 930us/step - loss: 0.6136 - accuracy: 0.6650
Epoch 88/100
46/46 [==============================] - 0s 909us/step - loss: 0.6121 - accuracy: 0.6692
Epoch 89/100
46/46 [==============================] - 0s 887us/step - loss: 0.6273 - accuracy: 0.6525
Epoch 90/100
46/46 [==============================] - 0s 887us/step - loss: 0.6044 - accuracy: 0.

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

16/16 - 0s - loss: 0.6389 - accuracy: 0.6351
Loss: 0.6389114856719971, Accuracy: 0.6350515484809875


# Optimization Attempt #2: Adding Neurons in Layers

In [40]:
# define model
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 30


nn = tf.keras.models.Sequential()

# first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# check structure of model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 90)                450       
_________________________________________________________________
dense_5 (Dense)              (None, 30)                2730      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 31        
Total params: 3,211
Trainable params: 3,211
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
46/46 [==============================] - 0s 842us/step - loss: 0.6903 - accuracy: 0.5480
Epoch 2/100
46/46 [==============================] - 0s 909us/step - loss: 0.6826 - accuracy: 0.5733
Epoch 3/100
46/46 [==============================] - 0s 842us/step - loss: 0.6764 - accuracy: 0.6211
Epoch 4/100
46/46 [==============================] - 0s 865us/step - loss: 0.6749 - accuracy: 0.6129
Epoch 5/100
46/46 [==============================] - 0s 931us/step - loss: 0.6658 - accuracy: 0.6075
Epoch 6/100
46/46 [==============================] - 0s 864us/step - loss: 0.6680 - accuracy: 0.6042
Epoch 7/100
46/46 [==============================] - 0s 953us/step - loss: 0.6596 - accuracy: 0.6356
Epoch 8/100
46/46 [==============================] - 0s 944us/step - loss: 0.6546 - accuracy: 0.6476
Epoch 9/100
46/46 [==============================] - 0s 931us/step - loss: 0.6560 - accuracy: 0.6417
Epoch 10/100
46/46 [==============================] - 0s 886us/step - loss: 0.6454 - accura

46/46 [==============================] - 0s 1ms/step - loss: 0.6105 - accuracy: 0.6814
Epoch 82/100
46/46 [==============================] - 0s 1ms/step - loss: 0.6336 - accuracy: 0.6422
Epoch 83/100
46/46 [==============================] - 0s 997us/step - loss: 0.6140 - accuracy: 0.6631
Epoch 84/100
46/46 [==============================] - 0s 886us/step - loss: 0.6109 - accuracy: 0.6730
Epoch 85/100
46/46 [==============================] - 0s 887us/step - loss: 0.6179 - accuracy: 0.6751
Epoch 86/100
46/46 [==============================] - 0s 887us/step - loss: 0.6237 - accuracy: 0.6627
Epoch 87/100
46/46 [==============================] - 0s 864us/step - loss: 0.6322 - accuracy: 0.6664
Epoch 88/100
46/46 [==============================] - 0s 842us/step - loss: 0.6207 - accuracy: 0.6565
Epoch 89/100
46/46 [==============================] - 0s 798us/step - loss: 0.6161 - accuracy: 0.6714
Epoch 90/100
46/46 [==============================] - 0s 820us/step - loss: 0.6227 - accuracy: 0.66

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

16/16 - 0s - loss: 0.6476 - accuracy: 0.6082
Loss: 0.6476240754127502, Accuracy: 0.6082473993301392


# Optmization Attempt #3 - Add Epochs

In [44]:
# define model
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# check structure of model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 80)                400       
_________________________________________________________________
dense_8 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 31        
Total params: 2,861
Trainable params: 2,861
Non-trainable params: 0
_________________________________________________________________


In [45]:
#compile model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
# train model
fit_model = nn.fit(X_train, y_train, epochs=150)

Epoch 1/150
46/46 [==============================] - 0s 864us/step - loss: 0.6900 - accuracy: 0.5350
Epoch 2/150
46/46 [==============================] - 0s 909us/step - loss: 0.6873 - accuracy: 0.5783
Epoch 3/150
46/46 [==============================] - 0s 909us/step - loss: 0.6814 - accuracy: 0.5238
Epoch 4/150
46/46 [==============================] - 0s 814us/step - loss: 0.6765 - accuracy: 0.5987
Epoch 5/150
46/46 [==============================] - 0s 842us/step - loss: 0.6740 - accuracy: 0.5883
Epoch 6/150
46/46 [==============================] - 0s 909us/step - loss: 0.6706 - accuracy: 0.6188
Epoch 7/150
46/46 [==============================] - 0s 953us/step - loss: 0.6594 - accuracy: 0.6187
Epoch 8/150
46/46 [==============================] - 0s 975us/step - loss: 0.6547 - accuracy: 0.6431
Epoch 9/150
46/46 [==============================] - 0s 886us/step - loss: 0.6529 - accuracy: 0.6398
Epoch 10/150
46/46 [==============================] - 0s 997us/step - loss: 0.6581 - accura

46/46 [==============================] - 0s 931us/step - loss: 0.6269 - accuracy: 0.6600
Epoch 83/150
46/46 [==============================] - 0s 1ms/step - loss: 0.6225 - accuracy: 0.6583
Epoch 84/150
46/46 [==============================] - 0s 1ms/step - loss: 0.6184 - accuracy: 0.6632
Epoch 85/150
46/46 [==============================] - 0s 1ms/step - loss: 0.6196 - accuracy: 0.6679
Epoch 86/150
46/46 [==============================] - 0s 1ms/step - loss: 0.6093 - accuracy: 0.6737
Epoch 87/150
46/46 [==============================] - 0s 1ms/step - loss: 0.6296 - accuracy: 0.6482
Epoch 88/150
46/46 [==============================] - 0s 953us/step - loss: 0.6232 - accuracy: 0.6560
Epoch 89/150
46/46 [==============================] - 0s 1ms/step - loss: 0.6294 - accuracy: 0.6505
Epoch 90/150
46/46 [==============================] - 0s 997us/step - loss: 0.6316 - accuracy: 0.6467
Epoch 91/150
46/46 [==============================] - 0s 1ms/step - loss: 0.6239 - accuracy: 0.6704
Epoch 9

In [47]:
# evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

16/16 - 0s - loss: 0.6367 - accuracy: 0.6103
Loss: 0.6367443203926086, Accuracy: 0.6103093028068542


# Optimization Attempt #4: Change Activation Function w/ Callback

In [49]:
# define checkpoint path and filenames
import os
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [50]:
# define model
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# output layer
nn.add(tf.keras.layers.Dense(units=1, activation='tanh'))

# check structure of model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 80)                400       
_________________________________________________________________
dense_11 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 31        
Total params: 2,861
Trainable params: 2,861
Non-trainable params: 0
_________________________________________________________________


In [53]:
from tensorflow.keras.callbacks import ModelCheckpoint

# compile model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# create callback that saves weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# train model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

# evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
46/46 [==============================] - 0s 821us/step - loss: 0.6304 - accuracy: 0.6479
Epoch 2/100
46/46 [==============================] - 0s 864us/step - loss: 0.6148 - accuracy: 0.6676
Epoch 3/100
46/46 [==============================] - 0s 887us/step - loss: 0.6205 - accuracy: 0.6792
Epoch 4/100
46/46 [==============================] - 0s 864us/step - loss: 0.6252 - accuracy: 0.6375
Epoch 5/100
46/46 [==============================] - 0s 866us/step - loss: 0.6254 - accuracy: 0.6615
Epoch 6/100
46/46 [==============================] - 0s 887us/step - loss: 0.6180 - accuracy: 0.6528
Epoch 7/100
46/46 [==============================] - 0s 820us/step - loss: 0.6214 - accuracy: 0.6723
Epoch 8/100
46/46 [==============================] - 0s 887us/step - loss: 0.6234 - accuracy: 0.6603
Epoch 9/100
46/46 [==============================] - 0s 776us/step - loss: 0.6108 - accuracy: 0.6680
Epoch 10/100
46/46 [==============================] - 0s 931us/step - loss: 0.6088 - accura

46/46 [==============================] - 0s 931us/step - loss: 0.6146 - accuracy: 0.6521
Epoch 78/100
46/46 [==============================] - 0s 909us/step - loss: 0.6110 - accuracy: 0.6684
Epoch 79/100
46/46 [==============================] - 0s 975us/step - loss: 0.6175 - accuracy: 0.6608
Epoch 80/100
46/46 [==============================] - 0s 909us/step - loss: 0.6179 - accuracy: 0.6504
Epoch 81/100
46/46 [==============================] - 0s 864us/step - loss: 0.6027 - accuracy: 0.6736
Epoch 82/100
46/46 [==============================] - 0s 909us/step - loss: 0.6054 - accuracy: 0.6798
Epoch 83/100
46/46 [==============================] - 0s 975us/step - loss: 0.6131 - accuracy: 0.6697
Epoch 84/100
46/46 [==============================] - 0s 865us/step - loss: 0.6020 - accuracy: 0.6682
Epoch 85/100
46/46 [==============================] - 0s 887us/step - loss: 0.6094 - accuracy: 0.6699
Epoch 86/100
46/46 [==============================] - 0s 909us/step - loss: 0.6203 - accuracy: 